In [1]:
import yfinance as yf
import pandas as pd

### investments tickers

In [2]:
symbol_type = "ETF"

In [3]:
yf_market_info  = pd.read_csv('../data/market_info.csv')

In [4]:
ticker_info_df = yf_market_info[yf_market_info.symbol_type==symbol_type].reset_index()
ticker_list = list(ticker_info_df.symbol)

In [21]:
ticker_symbols  = " ".join(ticker_list)

In [22]:
ticker_symbols = ticker_symbols + " EURGBP=X"

In [23]:
data = yf.download(ticker_symbols, period='1y')
data.head()

[*********************100%***********************]  5 of 5 completed


Price         Close                                                  High  \
Ticker       CNKY.L EURGBP=X     FTAL.L  S600.L     SXR8.DE        CNKY.L   
Date                                                                        
2024-01-15  19374.0      NaN  61.369999  9664.5  458.059998  19375.000000   
2024-01-16  19236.0  0.86061  61.080002  9627.0  460.459991  19239.000000   
2024-01-17  18732.0  0.86072  60.139999  9494.5  458.510010  18891.800781   
2024-01-18  19003.5  0.85862  60.259998  9547.5  460.089996  19046.000000   
2024-01-19  19117.0  0.85590  60.240002  9549.5  462.869995  19168.099609   

Price                                               ...     Open           \
Ticker     EURGBP=X     FTAL.L  S600.L     SXR8.DE  ...   CNKY.L EURGBP=X   
Date                                                ...                     
2024-01-15      NaN  61.820000  9692.0  458.579987  ...  19325.0      NaN   
2024-01-16  0.86190  61.369999  9635.0  460.929993  ...  19155.0  0.86061   
2024-01-17  0.86180  60.540001  9507.0  458.579987  ...  18886.0  0.86072   
2024-01-18  0.85910  60.376301  9531.0  460.369995  ...  18860.0  0.85862   
2024-01-19  0.85921  60.660000  9563.0  463.720001  ...  19100.0  0.85590   

Price                                      Volume                           \
Ticker         FTAL.L  S600.L     SXR8.DE  CNKY.L EURGBP=X   FTAL.L S600.L   
Date                                                                         
2024-01-15  61.439999  9692.0  458.040009  2850.0      NaN   3357.0  245.0   
2024-01-16  61.369999  9635.0  457.290009   358.0      0.0  22713.0   69.0   
2024-01-17  60.540001  9507.0  457.690002  1809.0      0.0   6163.0  112.0   
2024-01-18  59.919998  9531.0  457.049988   598.0      0.0   9457.0  106.0   
2024-01-19  60.500000  9563.0  462.100006   489.0      0.0   2487.0    3.0   

Price                
Ticker      SXR8.DE  
Date                 
2024-01-15  39147.0  
2024-01-16  19100.0  
2024-01-17  38310.0  
2024-01-18  22278.0  
2024-01-19  21492.0  

[5 rows x 25 columns]

In [8]:
data.columns

MultiIndex([( 'Close',   'CNKY.L'),
            ( 'Close',   'FTAL.L'),
            ( 'Close',   'S600.L'),
            ( 'Close',  'SXR8.DE'),
            ( 'Close', 'USDGBP=X'),
            (  'High',   'CNKY.L'),
            (  'High',   'FTAL.L'),
            (  'High',   'S600.L'),
            (  'High',  'SXR8.DE'),
            (  'High', 'USDGBP=X'),
            (   'Low',   'CNKY.L'),
            (   'Low',   'FTAL.L'),
            (   'Low',   'S600.L'),
            (   'Low',  'SXR8.DE'),
            (   'Low', 'USDGBP=X'),
            (  'Open',   'CNKY.L'),
            (  'Open',   'FTAL.L'),
            (  'Open',   'S600.L'),
            (  'Open',  'SXR8.DE'),
            (  'Open', 'USDGBP=X'),
            ('Volume',   'CNKY.L'),
            ('Volume',   'FTAL.L'),
            ('Volume',   'S600.L'),
            ('Volume',  'SXR8.DE'),
            ('Volume', 'USDGBP=X')],
           names=['Price', 'Ticker'])

### returns

In [9]:
yf_period = "1y" # ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
yf_interval = "1d"

In [10]:
#  1.  Create dataframe yf_price with yf.download
yf_returns = yf.download(
        tickers = ticker_symbols,       # tickers list or string as well
        period = yf_period,      # optional, default is '1mo'
        interval = yf_interval,  # fetch data by intervaal
        group_by = 'ticker',     # group by ticker
        auto_adjust = True,      # adjust all OHLC (open-high-low-close)
        prepost = True,          # download market hours data
        threads = True,          # threads for mass downloading
        proxy = None)            # proxy

#  2.  Select 'Close' (price at market close) column only
yf_returns = yf_returns.iloc[:, yf_returns.columns.get_level_values(1)=='Close']

#  3.  Remove the dataframe multi-index
yf_returns.columns = yf_returns.columns.droplevel(1)  # multi-index

yf_returns = round(yf_returns.pct_change()*100, 2)

yf_returns.tail(10)

[*********************100%***********************]  5 of 5 completed


Ticker,CNKY.L,USDGBP=X,S600.L,SXR8.DE,FTAL.L
Date,,,,,
2025-01-03,-0.37,1.12,-0.51,-0.06,-0.64
2025-01-06,-0.19,-0.38,1.14,0.53,0.21
2025-01-07,0.31,-0.67,0.26,-1.00,-0.21
2025-01-08,0.80,0.27,0.18,0.13,-0.23
2025-01-09,-0.48,0.95,0.82,0.08,0.83
2025-01-10,-1.03,0.49,-0.64,-0.98,-0.93
2025-01-13,-0.19,0.77,-0.52,-0.05,-0.30
2025-01-14,-0.14,-0.23,0.41,-0.22,-0.26
2025-01-15,0.78,0.00,1.13,1.75,1.43


### dividends - check all ETFs are acc

In [11]:
ticker_list

['SXR8.DE', 'FTAL.L', 'CNKY.L', 'S600.L']

In [12]:
yf_dividend = pd.DataFrame()   # initialize dataframe

for i in ticker_list:
    x = pd.DataFrame(yf.Ticker(i).dividends)
    x = x.rename(columns={"Dividends":i})
    yf_dividend = pd.concat([yf_dividend,x], axis=1)
    if len(x) > 0:
        print('{:>8}\t- dividends'.format(i))
    else:
        print('{:>8}\t- no dividends'.format(i))             

#  match dates in yf_returns (first return data to now)
yf_dividend = yf_dividend[yf_dividend.index >= yf_returns.index[0]]

#  print out dividends
print("\n",yf_dividend.tail(10))

 SXR8.DE	- no dividends
  FTAL.L	- no dividends
  CNKY.L	- no dividends
  S600.L	- no dividends


TypeError: Invalid comparison between dtype=datetime64[ns, UTC] and Timestamp

### check currencies

In [14]:
tickers = yf.Tickers(ticker_symbols)

In [15]:
tickers.tickers['SXR8.DE'].info

{'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 601.28,
 'open': 601.26,
 'dayLow': 601.18,
 'dayHigh': 612.26,
 'regularMarketPreviousClose': 601.28,
 'regularMarketOpen': 601.26,
 'regularMarketDayLow': 601.18,
 'regularMarketDayHigh': 612.26,
 'volume': 45365,
 'regularMarketVolume': 45365,
 'averageVolume': 74107,
 'averageVolume10days': 51701,
 'averageDailyVolume10Day': 51701,
 'ask': 611.94,
 'bidSize': 3200,
 'askSize': 4700,
 'fiftyTwoWeekLow': 456.78,
 'fiftyTwoWeekHigh': 615.6,
 'fiftyDayAverage': 604.1156,
 'twoHundredDayAverage': 549.3077,
 'currency': 'EUR',
 'fundFamily': 'BlackRock Asset Management Ireland - ETF',
 'fundInceptionDate': 1576022400,
 'legalType': 'Exchange Traded Fund',
 'exchange': 'GER',
 'quoteType': 'ETF',
 'symbol': 'SXR8.DE',
 'underlyingSymbol': 'SXR8.DE',
 'shortName': 'iShs VII-Core S&P 500 U.ETF   R',
 'longName': 'iShares Core S&P 500 UCITS ETF USD (Acc)',
 'firstTradeDateEpochUtc': 1274252400,
 'timeZoneFullName': 'Europe/Berlin',
 'timeZ

In [17]:
tickers.tickers['FTAL.L'].info

{'phone': '+353 1 776 3000',
 'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 67.98,
 'open': 67.84,
 'dayLow': 67.84,
 'dayHigh': 68.95,
 'regularMarketPreviousClose': 67.98,
 'regularMarketOpen': 67.84,
 'regularMarketDayLow': 67.84,
 'regularMarketDayHigh': 68.95,
 'trailingPE': 0.16081622,
 'volume': 2144,
 'regularMarketVolume': 2144,
 'averageVolume': 9056,
 'averageVolume10days': 14348,
 'averageDailyVolume10Day': 14348,
 'bid': 68.88,
 'ask': 68.98,
 'totalAssets': 485574816,
 'fiftyTwoWeekLow': 54.6595,
 'fiftyTwoWeekHigh': 70.6,
 'fiftyDayAverage': 68.4052,
 'twoHundredDayAverage': 67.902824,
 'navPrice': 67.87211,
 'currency': 'GBP',
 'ytdReturn': -0.0054133,
 'beta3Year': 0.98,
 'fundFamily': 'State Street Global Advisors Europe Limited',
 'fundInceptionDate': 1524700800,
 'legalType': 'Exchange Traded Fund',
 'threeYearAverageReturn': 0.0504992,
 'fiveYearAverageReturn': 0.042945698,
 'exchange': 'LSE',
 'quoteType': 'ETF',
 'symbol': 'FTAL.L',
 'underlyingSymbol': 'FT

In [18]:
tickers.tickers['CNKY.L'].info

{'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 20118.0,
 'open': 20045.0,
 'dayLow': 20045.0,
 'dayHigh': 20305.0,
 'regularMarketPreviousClose': 20118.0,
 'regularMarketOpen': 20045.0,
 'regularMarketDayLow': 20045.0,
 'regularMarketDayHigh': 20305.0,
 'volume': 1983,
 'regularMarketVolume': 1983,
 'averageVolume': 1238,
 'averageVolume10days': 473,
 'averageDailyVolume10Day': 473,
 'bid': 20267.0,
 'ask': 20283.0,
 'bidSize': 14400,
 'askSize': 514400,
 'totalAssets': 52038213632,
 'fiftyTwoWeekLow': 16000.0,
 'fiftyTwoWeekHigh': 21362.0,
 'fiftyDayAverage': 20137.47,
 'twoHundredDayAverage': 19759.21,
 'navPrice': 39355.52,
 'currency': 'GBp',
 'fundFamily': 'BlackRock Asset Management Ireland - ETF',
 'fundInceptionDate': 1264377600,
 'legalType': 'Exchange Traded Fund',
 'exchange': 'LSE',
 'quoteType': 'ETF',
 'symbol': 'CNKY.L',
 'underlyingSymbol': 'CNKY.L',
 'shortName': 'ISHARES VII PLC ISHRS NIKKEI 22',
 'longName': 'iShares VII PLC - iShares Nikkei 225 ETF JPY Acc',
 

In [19]:
tickers.tickers['S600.L'].info

{'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 10436.0,
 'open': 10460.212,
 'dayLow': 10460.212,
 'dayHigh': 10560.0,
 'regularMarketPreviousClose': 10436.0,
 'regularMarketOpen': 10460.212,
 'regularMarketDayLow': 10460.212,
 'regularMarketDayHigh': 10560.0,
 'volume': 902,
 'regularMarketVolume': 902,
 'averageVolume': 1932,
 'averageVolume10days': 1258,
 'averageDailyVolume10Day': 1258,
 'bid': 10554.0,
 'ask': 10558.0,
 'totalAssets': 462375008,
 'fiftyTwoWeekLow': 9504.864,
 'fiftyTwoWeekHigh': 10864.0,
 'fiftyDayAverage': 10308.94,
 'twoHundredDayAverage': 10478.415,
 'currency': 'GBp',
 'fundFamily': 'Invesco Investment Management Limited',
 'fundInceptionDate': 1238544000,
 'legalType': 'Exchange Traded Fund',
 'exchange': 'LSE',
 'quoteType': 'ETF',
 'symbol': 'S600.L',
 'underlyingSymbol': 'S600.L',
 'shortName': 'INVESCO MARKETS PLC INVESCO STO',
 'longName': 'Invesco STOXX Europe 600 UCITS ETF',
 'firstTradeDateEpochUtc': 1404370800,
 'timeZoneFullName': 'Europe/Lond

In [16]:
ticker_info_df

,index,symbol,symbol_name,symbol_type,symbol_name_full,unit,website
0,4,SXR8.DE,S&P500,ETF,iShares Core S&P 500 UCITS ETF USD (Acc),EUR,NaN
1,12,FTAL.L,FTSE all-share,ETF,SPDR FTSE UK All Share UCITS ETF Acc,GBP,NaN
2,15,CNKY.L,Nikkei 225,ETF,iShares VII PLC - iShares Nikkei 225 ETF JPY Acc,NaN,NaN
3,17,S600.L,Stoxx 600,ETF,Invesco ETF (S600.L),GBP,https://stoxx.com/index/sxxp/
